<a href="https://colab.research.google.com/github/cnrgrl/PANDAS/blob/main/08_Filtering_with_the_query_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment the following line, if you are using google collab
!rm -r Pandas
!git clone https://github.com/Wuebbelt/Pandas.git

Cloning into 'Pandas'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 83 (delta 14), reused 80 (delta 11), pack-reused 0
Unpacking objects: 100% (83/83), done.


# Filtering with the `query` Method

The previous chapters on boolean selection showed us how to filter our DataFrames and Series based on their values. We created conditions, usually involving the comparison operators, resulting in boolean Series and passed them to *just the brackets* to filter the data.

In this chapter we cover the `query` method which enables us to also make selections based on the values of the DataFrame or Series. The `query` method is easier and more intuitive to use than boolean selection, but doesn't provide as much functionality to filter the data. Still, it is a good method to know about to make your subset selections more readable.

## The `query` method

The `query` method allows you to filter the data by writing the condition as a string. For instance, you would pass the string `'tripduration > 1000'` to select all rows of the `bikes` dataset that have a `tripduration` less than 1000. Let's read in the bikes dataset and run this command now.

In [ ]:
import pandas as pd
bikes = pd.read_csv('Pandas/bikes.csv', parse_dates=['starttime', 'stoptime'])
bikes.head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
0,7147,Subscriber,Male,2013-06-28 19:01:00,2013-06-28 19:17:00,993,Lake Shore Dr & Monroe St,41.881050,-87.616970,11.0,Michigan Ave & Oak St,41.90096,-87.623777,15.0,73.9,10.0,12.7,-9999.0,mostlycloudy
1,7524,Subscriber,Male,2013-06-28 22:53:00,2013-06-28 23:03:00,623,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wells St & Walton St,41.89993,-87.634430,19.0,69.1,10.0,6.9,-9999.0,partlycloudy
2,10927,Subscriber,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,Dearborn St & Monroe St,41.88132,-87.629521,23.0,73.0,10.0,16.1,-9999.0,mostlycloudy


In [ ]:
bikes.query('tripduration > 1000').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
2,10927,Subscriber,Male,2013-06-30 14:43:00,2013-06-30 15:01:00,1040,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,Dearborn St & Monroe St,41.881320,-87.629521,23.0,73.0,10.0,16.1,-9999.0,mostlycloudy
8,21028,Subscriber,Male,2013-07-03 15:21:00,2013-07-03 15:42:00,1300,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Wood St & Division St,41.903320,-87.672730,15.0,71.1,8.0,0.0,-9999.0,cloudy
10,24383,Subscriber,Male,2013-07-04 17:17:00,2013-07-04 17:42:00,1523,Morgan St & 18th St,41.858086,-87.651073,15.0,Damen Ave & Pierce Ave,41.909396,-87.677692,19.0,79.0,10.0,9.2,-9999.0,mostlycloudy


### Less syntax and more readable

The `query` method generally uses less syntax than boolean selection and is usually more readable. For instance, the following reproduces the last result with boolean selection:

```
bikes[bikes['tripduration'] > 1000]
```

This looks a bit clumsy with the name `bikes` written twice right next to one another. The `query` method has its own set of rules for constitutes a correctly written condition within the string you pass it. The rest of this chapter covers all of the available functionality of the `query` method. This syntax only works within the `query` method and is not allowed anywhere else in pandas.

### Use strings `and`, `or`, `not`

Unlike boolean selection, you can use the strings `and`, `or`, and `not` instead of the operators which further aides readability with `query`. Let's select all rides with `tripduration` greater than 1,000 and `temperature` greater than 85.

In [ ]:
bikes.query('tripduration > 1000 and temperature > 85').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
40,61401,Subscriber,Female,2013-07-14 14:08:00,2013-07-14 15:53:00,6274,Wabash Ave & Roosevelt Rd,41.867173,-87.625955,19.0,Lake Shore Dr & Monroe St,41.881050,-87.616970,11.0,87.1,10.0,8.1,-9999.0,partlycloudy
53,68864,Subscriber,Male,2013-07-16 13:04:00,2013-07-16 13:28:00,1435,Canal St & Jackson Blvd,41.878114,-87.639971,35.0,Canal St & Jackson Blvd,41.878114,-87.639971,35.0,90.0,10.0,8.1,-9999.0,mostlycloudy
60,71812,Subscriber,Male,2013-07-17 10:23:00,2013-07-17 10:40:00,1024,Clinton St & Washington Blvd,41.883380,-87.641170,31.0,Larrabee St & Menomonee St,41.914680,-87.643320,15.0,88.0,10.0,5.8,-9999.0,partlycloudy


### Chained comparisons

Let's say we want to find all rides where the temperature was between 50 and 60 degrees. You can do this with query by using the and operator.

In [ ]:
bikes.query('temperature >= 50 and temperature <= 60').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
590,483022,Subscriber,Female,2013-09-13 07:55:00,2013-09-13 08:01:00,319,Greenview Ave & Fullerton Ave,41.925330,-87.665800,15.0,Sheffield Ave & Fullerton Ave,41.925602,-87.653708,15.0,54.0,10.0,15.0,-9999.0,partlycloudy
591,483128,Subscriber,Male,2013-09-13 08:04:00,2013-09-13 08:16:00,738,Lincoln Ave & Armitage Ave,41.918273,-87.638116,19.0,Larrabee St & Kingsbury St,41.897764,-87.642884,27.0,57.9,10.0,13.8,-9999.0,partlycloudy
592,483127,Subscriber,Female,2013-09-13 08:04:00,2013-09-13 08:14:00,599,Orleans St & Elm St,41.902924,-87.637715,15.0,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,57.9,10.0,13.8,-9999.0,partlycloudy


While this syntax is valid, there is a better way. You can use a **chained comparison** to make the string even more readable and concise. A chained comparison places the column name between two comparison operators. The following implies that 50 is less than or equal to the temperature and the temperature is less than or equal to 60 which is equivalent to our previous selection.

In [ ]:
bikes.query('50 <= temperature <= 60').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
590,483022,Subscriber,Female,2013-09-13 07:55:00,2013-09-13 08:01:00,319,Greenview Ave & Fullerton Ave,41.925330,-87.665800,15.0,Sheffield Ave & Fullerton Ave,41.925602,-87.653708,15.0,54.0,10.0,15.0,-9999.0,partlycloudy
591,483128,Subscriber,Male,2013-09-13 08:04:00,2013-09-13 08:16:00,738,Lincoln Ave & Armitage Ave,41.918273,-87.638116,19.0,Larrabee St & Kingsbury St,41.897764,-87.642884,27.0,57.9,10.0,13.8,-9999.0,partlycloudy
592,483127,Subscriber,Female,2013-09-13 08:04:00,2013-09-13 08:14:00,599,Orleans St & Elm St,41.902924,-87.637715,15.0,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,57.9,10.0,13.8,-9999.0,partlycloudy


### Reference strings with quotes

If you would like to reference a literal string within `query`, you need to surround it in quotes, or else pandas will attempt to use it as a column name. Let's select all rides done by a 'Female' with a trip duration greater than 2,000.

In [ ]:
bikes.query('gender == "Female" and tripduration > 2000').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
40,61401,Subscriber,Female,2013-07-14 14:08:00,2013-07-14 15:53:00,6274,Wabash Ave & Roosevelt Rd,41.867173,-87.625955,19.0,Lake Shore Dr & Monroe St,41.881050,-87.616970,11.0,87.1,10.0,8.1,-9999.0,partlycloudy
77,87005,Subscriber,Female,2013-07-21 11:35:00,2013-07-21 13:54:00,8299,State St & 19th St,41.856594,-87.627542,15.0,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,82.9,10.0,5.8,-9999.0,mostlycloudy
173,178550,Subscriber,Female,2013-08-08 08:49:00,2013-08-08 09:31:00,2502,Sheffield Ave & Addison St,41.946880,-87.654450,27.0,Dearborn St & Adams St,41.879356,-87.629791,19.0,71.1,10.0,10.4,-9999.0,mostlycloudy


### Forgetting quotes

If you do not use quotes around your literal string then pandas assumes that value is a column name. The following raises an error. It believes you are accessing a column name Female, but that doesn't exist.

In [ ]:
bikes.query('gender == Female and tripduration > 2000')

UndefinedVariableError: ignored

### Column to column comparisons

It is possible to compare each value in one column with each value in another column. Here, we filter for all the rides where there were more bikes at the start than at the end.

In [ ]:
bikes.query('dpcapacity_start > dpcapacity_end').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
1,7524,Subscriber,Male,2013-06-28 22:53:00,2013-06-28 23:03:00,623,Clinton St & Washington Blvd,41.883380,-87.64117,31.0,Wells St & Walton St,41.89993,-87.63443,19.0,69.1,10.0,6.9,-9999.0,partlycloudy
6,18880,Subscriber,Male,2013-07-02 17:47:00,2013-07-02 17:56:00,565,Clark St & Randolph St,41.884576,-87.63189,31.0,Ravenswood Ave & Irving Park Rd,41.95469,-87.67393,19.0,66.0,10.0,15.0,-9999.0,cloudy
8,21028,Subscriber,Male,2013-07-03 15:21:00,2013-07-03 15:42:00,1300,Clinton St & Washington Blvd,41.883380,-87.64117,31.0,Wood St & Division St,41.90332,-87.67273,15.0,71.1,8.0,0.0,-9999.0,cloudy


### Use 'in' for multiple equalities

You can check whether each value in a column is equal to one or more other values by using the word 'in' within your query. Use the syntax for creating a list withing the query string to contain all the values you'd like to check. The following tests whether the ride weather event was snow or rain.

In [ ]:
bikes.query('events in ["snow", "rain"]').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
45,66336,Subscriber,Male,2013-07-15 16:43:00,2013-07-15 16:55:00,727,Greenwood Ave & 47th St,41.809835,-87.599383,15.0,State St & Harrison St,41.873958,-87.627739,19.0,82.9,10.0,5.8,0.0,rain
112,111568,Subscriber,Male,2013-07-26 19:10:00,2013-07-26 19:33:00,1395,Larrabee St & Kingsbury St,41.897764,-87.642884,27.0,Damen Ave & Pierce Ave,41.909396,-87.677692,19.0,66.9,8.0,12.7,0.0,rain
124,130156,Subscriber,Male,2013-07-30 18:53:00,2013-07-30 19:00:00,442,Canal St & Jackson Blvd,41.878114,-87.639971,35.0,Racine Ave & Congress Pkwy,41.874640,-87.657030,19.0,69.1,10.0,3.5,0.0,rain


There are multiple syntaxes for the above that all work the same, but I prefer using the above as it is most similar to the `isin` method used during boolean selection.

* `bikes.query('["snow", "rain"] in events')`
* `bikes.query('["snow", "rain"] == events')`
* `bikes.query('events == ["snow", "rain"]')`

### Use 'not in' to invert the condition

You can invert the result of an 'in' clause by placing the word 'not' before it. Here, we find all the rides that did not have the weather events cloudy, partly cloudy or mostly cloudy.

In [ ]:
bikes.query('events not in ["cloudy", "partlycloudy", "mostlycloudy"]').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
25,47798,Subscriber,Female,2013-07-11 08:17:00,2013-07-11 08:31:00,830,Wabash Ave & Roosevelt Rd,41.867173,-87.625955,19.0,Daley Center Plaza,41.884337,-87.630183,47.0,73.9,10.0,8.1,-9999.0,clear
26,51130,Subscriber,Male,2013-07-12 01:07:00,2013-07-12 01:24:00,1043,State St & Harrison St,41.873958,-87.627739,19.0,Racine Ave & 18th St,41.858181,-87.656487,15.0,64.9,10.0,0.0,-9999.0,clear
33,53963,Subscriber,Male,2013-07-12 17:22:00,2013-07-12 17:34:00,730,Clark St & Congress Pkwy,41.875933,-87.630585,27.0,Racine Ave & Congress Pkwy,41.874640,-87.657030,19.0,79.0,10.0,10.4,-9999.0,clear


### Arithmetic operations within `query`

It is possible to write arithmetic operations within `query` just as you would outside of it. For instance, if we wanted to find all the rides such that there were 20 more bikes at the start station than at the end, we do the following.

In [ ]:
bikes.query('dpcapacity_start - dpcapacity_end > 20').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
116,112863,Subscriber,Male,2013-07-27 09:54:00,2013-07-27 09:56:00,121,Daley Center Plaza,41.884337,-87.630183,47.0,LaSalle St & Washington St,41.882664,-87.632530,15.0,60.8,10.0,11.5,-9999.0,cloudy
296,289374,Subscriber,Male,2013-08-22 08:54:00,2013-08-22 09:02:00,478,Daley Center Plaza,41.884337,-87.630183,47.0,Halsted St & Madison St,41.881750,-87.647800,19.0,72.0,10.0,10.4,0.0,cloudy
548,463388,Subscriber,Female,2013-09-10 17:04:00,2013-09-10 17:15:00,651,Daley Center Plaza,41.884337,-87.630183,47.0,Kingsbury St & Erie St,41.893843,-87.641851,23.0,93.0,10.0,13.8,-9999.0,partlycloudy


### Filtering for right triangles

Let's read in the triangles dataset which contains the lengths of each side of a triangle as the columns `a`, `b`, and `c`.

In [ ]:
triangles = pd.read_csv('Pandas/triangles.csv')
triangles.head()

,a,b,c
0,2,3,4
1,3,2,4
2,3,4,5
3,3,5,6
4,3,6,7


We can use the `query` method to find all the right triangles, those that satisfy the Pythagorean Theorem. We write the condition using the arithmetic and comparison operators.

In [ ]:
triangles.query('a ** 2 + b ** 2 == c ** 2').head()

,a,b,c
2,3,4,5
5,4,3,5
14,5,12,13
21,6,8,10
33,7,24,25


The syntax is quite a bit nicer than the boolean selection alternative.

In [ ]:
filt = triangles['a'] ** 2 + triangles['b'] ** 2 == triangles['c'] ** 2
triangles[filt].head()

,a,b,c
2,3,4,5
5,4,3,5
14,5,12,13
21,6,8,10
33,7,24,25


### Use the `@` symbol to reference a variable name

By default, all words within the query string attempt to reference the column name. You can, however, reference a variable name by preceding it with the `@` symbol. Let's assign the variable name `min_length` to 5000 and reference it in a query to find all the rides where trip duration was greater than it.

In [ ]:
min_length = 5000
bikes.query('tripduration > @min_length').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
18,40924,Subscriber,Male,2013-07-09 13:12:00,2013-07-09 14:42:00,5396,Canal St & Jackson Blvd,41.878114,-87.639971,35.0,Millennium Park,41.881032,-87.624084,35.0,79.0,10.0,13.8,0.0,cloudy
40,61401,Subscriber,Female,2013-07-14 14:08:00,2013-07-14 15:53:00,6274,Wabash Ave & Roosevelt Rd,41.867173,-87.625955,19.0,Lake Shore Dr & Monroe St,41.881050,-87.616970,11.0,87.1,10.0,8.1,-9999.0,partlycloudy
77,87005,Subscriber,Female,2013-07-21 11:35:00,2013-07-21 13:54:00,8299,State St & 19th St,41.856594,-87.627542,15.0,Sheffield Ave & Kingsbury St,41.909592,-87.653497,15.0,82.9,10.0,5.8,-9999.0,mostlycloudy


### Using the index with `query`

You can even use the word `index` to make comparisons against the index as if it were a normal column. In the bikes DataFrame, the index is just the integers beginning at 0. Here, we select only the `events` that were 'cloudy' for an index value greater than 4000.

In [ ]:
bikes.query('index > 4000 and events == "cloudy" ').head(3)

,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_name,latitude_end,longitude_end,dpcapacity_end,temperature,visibility,wind_speed,precipitation,events
4007,2003400,Subscriber,Male,2014-06-07 14:07:00,2014-06-07 14:31:00,1434,Lake Shore Dr & North Blvd,41.911722,-87.626804,15.0,Halsted St & Roscoe St,41.943662,-87.648917,15.0,82.0,10.0,13.8,-9999.0,cloudy
4008,2004978,Subscriber,Male,2014-06-07 14:58:00,2014-06-07 15:19:00,1258,Theater on the Lake,41.926277,-87.630834,15.0,Sheridan Rd & Buena Ave,41.958400,-87.654230,15.0,82.0,10.0,13.8,-9999.0,cloudy
4009,2005778,Subscriber,Male,2014-06-07 15:23:00,2014-06-07 15:28:00,297,Sheffield Ave & Addison St,41.946880,-87.654450,27.0,Pine Grove Ave & Waveland Ave,41.949275,-87.646303,23.0,80.1,10.0,13.8,-9999.0,cloudy


### Using column names with spaces

pandas allows DataFrames to have column names with spaces in them. In order to use a column name containing spaces within `query`, you'll need to surround it with back ticks. If you don't use the back ticks you'll get an error. Let's read in the San Francisco employee compensation dataset which contains multiple column names that have spaces.

In [ ]:
sf_emp = pd.read_csv('Pandas/sf_employee_compensation.csv')
sf_emp.head(3)

,year,organization group,job,salaries,overtime,other salaries,retirement,health and dental,other benefits
0,2013,Public Protection,Personnel Technician,71414.01,0.00,0.0,14038.58,12918.24,5872.04
1,2013,General Administration & Finance,Planner 2,67941.06,0.00,0.0,13030.23,10047.52,5608.37
2,2013,Public Protection,Firefighter,116956.72,59975.43,19037.3,24796.44,15788.97,3222.20


Let's find all the employees that are in the organization group of 'Public Protection'.

In [ ]:
sf_emp.query('`organization group` == "Public Protection"').head(3)

,year,organization group,job,salaries,overtime,other salaries,retirement,health and dental,other benefits
0,2013,Public Protection,Personnel Technician,71414.01,0.00,0.0,14038.58,12918.24,5872.04
2,2013,Public Protection,Firefighter,116956.72,59975.43,19037.3,24796.44,15788.97,3222.20
7,2013,Public Protection,Police Officer,78591.02,2050.18,832.9,15383.49,11004.42,4471.61


### Selecting columns with `query`

Unfortunately the `query` method does not give us the ability to select a subset of the columns when filtering the data. You would have to do normal column selection after calling the method. Here, we use *just the brackets* to select three columns after finding all the rides where the weather was snow or rain.

In [ ]:
cols = ['starttime', 'temperature', 'events']
bikes.query('events in ["snow", "rain"]')[cols].head()

,starttime,temperature,events
45,2013-07-15 16:43:00,82.9,rain
112,2013-07-26 19:10:00,66.9,rain
124,2013-07-30 18:53:00,69.1,rain
161,2013-08-05 17:09:00,68.0,rain
498,2013-09-07 16:09:00,81.0,rain


## Summary

The `query` method provides an alternative to boolean selection to filter the data based on the values. Here are the rules for the string you provide.

* The expression in the string must evaluate as True or False for every row
* Column names may be accessed directly with their name
* Often you will use one of the comparison operators to create a condition
* Use `and`, `or`, and `not` to create more complex conditions
* To use a literal string, surround it with quotes
* Use chained comparison operators to shorten syntax
* Use `in` to test multiple equalities. Provide the test values in a list
* All arithmetic operators work just as they do outside of the string
* Use the `@` character to reference a variable name
* Use back ticks to reference a column name with spaces in it

## Exercises

Use the bikes dataset for the first few exercises.

### Exercise 1

<span  style="color:green; font-size:16px">Use the `query` method to select trip durations between 5000 and 10000.</span>

### Exercise 2

<span  style="color:green; font-size:16px">Use the `query` method to select trip durations between 5000 and 10000 when the weather was snow or rain. Retrieve the same data with boolean selection.</span>

### Exercise 3

<span  style="color:green; font-size:16px">Use the `query` method to select trip durations between 5000 and 10000 when it was snow or rain. Create a list outside of the query method to hold the weather and reference that variable with `@` within query.</span>

Read in the movie dataset by executing the cell below and use it for the following exercises.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
movie = pd.read_csv('Pandas/movie.csv', index_col='title')
movie.head(3)

,year,color,content_rating,duration,director_name,director_fb,actor1,actor1_fb,actor2,actor2_fb,actor3,actor3_fb,gross,genres,num_reviews,num_voted_users,plot_keywords,language,country,budget,imdb_score
title,,,,,,,,,,,,,,,,,,,,,
Avatar,2009.0,Color,PG-13,178.0,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,723.0,886204,avatar|future|marine|native|paraplegic,English,USA,237000000.0,7.9
Pirates of the Caribbean: At World's End,2007.0,Color,PG-13,169.0,Gore Verbinski,563.0,Johnny Depp,40000.0,Orlando Bloom,5000.0,Jack Davenport,1000.0,309404152.0,Action|Adventure|Fantasy,302.0,471220,goddess|marriage ceremony|marriage proposal|pi...,English,USA,300000000.0,7.1
Spectre,2015.0,Color,PG-13,148.0,Sam Mendes,0.0,Christoph Waltz,11000.0,Rory Kinnear,393.0,Stephanie Sigman,161.0,200074175.0,Action|Adventure|Thriller,602.0,275868,bomb|espionage|sequel|spy|terrorist,English,UK,245000000.0,6.8


### Exercise 4

<span  style="color:green; font-size:16px">Use the `query` method to find all movies where the total number of Facebook likes for all three actors is greater than 50,000.</span>

### Exercise 5

<span  style="color:green; font-size:16px">Select all the movies the number of user voters is less than 10 times the number of reviews.</span>

### Exercise 6

<span  style="color:green; font-size:16px">Select all the movies made in the 1990s that were rated R with an IMDB score greater than 8.</span>